In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("D:\Retail_store2.csv")

# Time based pricing

In [4]:
df['Date'] = pd.to_datetime(df['Date'])


In [5]:
df['DayOfWeek'] = df['Date'].dt.dayofweek
df['DayOfMonth'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month

In [6]:
# Create DayName for readability
day_map = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}
df['DayName'] = df['DayOfWeek'].map(day_map)

In [7]:
df['Time Adjustment'] = 0.0
# Weekend Increase (+5%)
# saturday(5) and sunday(6)
weekend_mask = df['DayOfWeek'].isin([5,6])
weekend_adj = 0.05
df.loc[weekend_mask, 'Time Adjustment'] += 0.05

In [8]:
# Peak season Increase (+5%)
peak_season_mask = df['Seasonality'].isin(['Autumn','Winter'])
season_adj = 0.05
df.loc[peak_season_mask, 'Time Adjustment'] += 0.05

In [9]:
# Month-end Increase (+2%)
month_end_mask = df['DayOfMonth'] > 25
month_end_adj = 0.02
df.loc[month_end_mask, 'Time Adjustment'] += 0.02

In [10]:
# Low demand Discount (-5%)
low_demand_threshold = df['Demand Forecast'].quantile(0.25)
low_demand_mask = df['Demand Forecast'] < low_demand_threshold
low_demand_adj = -0.05
df.loc[low_demand_mask, 'Time Adjustment'] += -0.05
print(f"Low Demand Threshold (25th percentuke) : {low_demand_threshold}")

Low Demand Threshold (25th percentuke) : 53.67


In [11]:
# Calcualte Total Adjustment Factor
df['Price Adjustment Factor'] = 0.0

# Apply adjustment
df.loc[weekend_mask,'Price Adjustment Factor'] += weekend_adj
df.loc[peak_season_mask,'Price Adjustment Factor'] += season_adj
df.loc[month_end_mask,'Price Adjustment Factor'] += month_end_adj
df.loc[low_demand_mask,'Price Adjustment Factor'] += low_demand_adj

In [12]:
# Calculate Final Price
df['Final Price'] = df['Price']*(1 + df['Price Adjustment Factor'])

# Display sample results
cols_to_show = ['Date','DayName','Seasonality','Demand Forecast','Price','Price Adjustment Factor','Final Price']
print(df[cols_to_show].sample(10).sort_values(by='Date'))

#save the results
df.to_csv('Retail_store_adjusted_pricing.csv',index=False)

            Date DayName Seasonality  Demand Forecast  Price  \
1363  2022-01-14     Fri      Winter            87.43  62.90   
7742  2022-03-19     Sat      Spring           281.21  25.45   
7876  2022-03-20     Sun      Summer            79.43  25.74   
10707 2022-04-18     Mon      Winter            19.02  85.13   
19360 2022-07-13     Wed      Autumn            70.05  19.23   
26969 2022-09-27     Tue      Winter            64.31  35.19   
37348 2023-01-09     Mon      Autumn           265.16  76.07   
59861 2023-08-22     Tue      Winter           151.83  10.60   
62786 2023-09-20     Wed      Winter            42.24  65.68   
71440 2023-12-16     Sat      Autumn            32.15  23.97   

       Price Adjustment Factor  Final Price  
1363                      0.05      66.0450  
7742                      0.05      26.7225  
7876                      0.05      27.0270  
10707                     0.00      85.1300  
19360                     0.05      20.1915  
26969              

# Define Inventory - Based Pricing Rules

In [13]:
df['Inventory Adjustment'] = 0.0

#  Define Inventory Thresholds based on percentiles
low_stock_threshold = df['Inventory Level'].quantile(0.10)
high_stock_threshold = df['Inventory Level'].quantile(0.75)
overstock_threshold = df['Inventory Level'].quantile(0.90)

In [14]:
#Initialize Inventory Adjustment Factor
df['Inventory Adjustment Factor'] = 0.0
df['Inventory Status'] = 'Normal'

In [15]:
# Low stock -> Increase Price (+10%)
low_stock_mask = df['Inventory Level'] < low_stock_threshold
df.loc[low_stock_mask,'Inventory Adjustment Factor'] = 0.10
df.loc[low_stock_mask,'Inventory Status'] = 'Low Stock'


In [16]:
# High Stock -> Decrease Price (-5%)
high_stock_mask = (df['Inventory Level'] >= high_stock_threshold) & (df['Inventory Level'] < overstock_threshold)
df.loc[high_stock_mask,'Inventory Adjustment Factor'] = -0.05
df.loc[high_stock_mask,'Inventory Status'] = 'High Stock'

In [17]:
# Overstocked -> Clearance Pricing (-15%)
overstock_mask = df['Inventory Level'] >= overstock_threshold
df.loc[overstock_mask,'Inventory Adjustment Factor'] = -0.15
df.loc[overstock_mask,'Inventory Status'] = 'Overstock'

In [18]:
# Calculation Final Price
df['Inventory Adjusted Price'] = df['Price'] * (1 + df['Inventory Adjustment Factor'])


In [19]:
# Select columns to display
cols = ['Date', 'Product ID', 'Inventory Level', 'Inventory Status', 'Price', 'Inventory Adjustment Factor', 'Inventory Adjusted Price']


# Print a sample sorted by Inventory Level to show different statuses
print(df[cols].sample(10).sort_values(by='Inventory Level'))

            Date Product ID  Inventory Level Inventory Status  Price  \
51367 2023-05-29      P0008               79        Low Stock  54.65   
35003 2022-12-17      P0004               84        Low Stock  18.25   
1378  2022-01-14      P0019              119           Normal  28.18   
8238  2022-03-24      P0019              147           Normal  95.78   
67751 2023-11-09      P0012              198           Normal  86.79   
51879 2023-06-03      P0020              243           Normal  52.86   
26992 2022-09-27      P0013              267           Normal  31.95   
49224 2023-05-08      P0005              269           Normal  47.06   
1957  2022-01-20      P0018              273           Normal  91.94   
72453 2023-12-26      P0014              405       High Stock  77.65   

       Inventory Adjustment Factor  Inventory Adjusted Price  
51367                         0.10                   60.1150  
35003                         0.10                   20.0750  
1378              

# 3. Generate rule_price column 

In [20]:

# Initialize with base price

df['rule_price'] = df['Price']

# Apply Time-Based Rules first
df['rule_price'] = df['rule_price'] * (1 + df['Time Adjustment'])


# Apply Inventory-Based Rules
df['rule_price'] = df['rule_price'] * (1 + df['Inventory Adjustment'])

# Round to 2 decimal places
df['rule_price'] = df['rule_price'].round(2)

# Save the final dataset
cols = ['Date', 'Price', 'Time Adjustment', 'Inventory Adjustment', 'rule_price']
print(df[cols].sample(5))

            Date  Price  Time Adjustment  Inventory Adjustment  rule_price
57583 2023-07-30  15.83             0.07                   0.0       16.94
63583 2023-09-28  61.63             0.02                   0.0       62.86
71744 2023-12-19  16.59             0.05                   0.0       17.42
37036 2023-01-06  43.89             0.00                   0.0       43.89
46604 2023-04-12  76.78             0.05                   0.0       80.62


# 4. Calculate Revenue Under Both Pricing Methods

In [21]:
static_revenue = (df['Price'] * df['Units Sold']).sum()

rule_revenue = (df['rule_price'] * df['Units Sold']).sum()

# Calculate Difference
diff = rule_revenue - static_revenue
pct_change = (diff/static_revenue) * 100

 
# Print Results
print("-"*30)
print(f"Total Static Revenue:     ${static_revenue:,.2f}")
print(f"Total Rule-Based Revenue: ${rule_revenue:,.2f}")
print(f"Net Difference:           ${diff:,.2f}")
print(f"Percentage Change:        ${pct_change:.2f}%")
print("-"*30)

------------------------------
Total Static Revenue:     $550,228,884.91
Total Rule-Based Revenue: $572,688,156.24
Net Difference:           $22,459,271.33
Percentage Change:        $4.08%
------------------------------


# 5. Compute Revenue Lift

In [30]:
# Calculate Revenues
df['Static Revenue'] = df['Price'] * df['Units Sold']
df['Rule-Based Revenue'] = df['rule_price'] * df['Units Sold']

# Compute Revenue Lift
total_static = df['Static Revenue'].sum()
total_dynamic = df['Rule-Based Revenue'].sum()
revenue_lift = (total_dynamic - total_static) / total_static

# Output Results
print("-"*30)
print(f"Total Static Revenue:        ${total_static:,.2f}")
print(f"Total Rule-Based Revenue:    ${total_dynamic:,.2f}")
print(f"Revenue Lift:                  {revenue_lift:.4%}")

if revenue_lift > 0:
    print("SUCCESS: Positive uplift indicates successful implementation.")

else:
    print('REVIEW: No positive uplift observed.')

------------------------------
Total Static Revenue:        $550,228,884.91
Total Rule-Based Revenue:    $572,688,156.24
Revenue Lift:                  4.0818%
SUCCESS: Positive uplift indicates successful implementation.
